In [1]:
# imports
import cv2
from concurrent.futures import ProcessPoolExecutor
import threading

# utils

import sys
import os

workspace_dir = os.path.abspath(os.path.join('workspace'))

if workspace_dir not in sys.path:
    sys.path.append(workspace_dir)

from resources import utils
from resources.features import extract_features, extract_features_from_patches
import joblib

In [2]:
svm, scaler, pca = joblib.load(os.getcwd()+'/presets/Preset4.pkl')

In [3]:
# step 1: process the folder images and save them in a temporary folder.
folder_dir = '/media/michalis/T/2-274-03-00788'

temp_dir = os.getcwd() + '/temp_images/' + os.path.basename(folder_dir)

os.makedirs(temp_dir, exist_ok=True)

print(temp_dir)

#utils.process_images(folder_dir, temp_dir, 0)

/home/michalis/Documents/Work/MELETI/inspection_client/temp_images/2-274-03-00788


In [4]:
# step 2: divide to patches and inspect every patch
def read_divide_classify(image_path):
    patches = utils.divide_into_patches(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE), (8,8))
    patch_features = extract_features_from_patches(patches)

    # for patch_touple in patch_features:
    #     plt.imshow(patch_touple[0], cmap='gray')
    #     plt.show()
    
    # index = 28
    # plt.imshow(patch_features[index][0], cmap='gray', vmin=0, vmax=255)
    # plt.show()

    # print(pca.transform([patch_features[index][1]]))

    labels = svm.predict(pca.transform(scaler.transform([patch_features[i][1] for i in range(32)])))

    image_count = os.path.basename(image_path)[:4]

    labeled_patches = [(patch_features[i][0], image_count, labels[i]) for i in range(32)]

    return labeled_patches

image_paths = os.listdir(temp_dir)

image_path = temp_dir + '/' + image_paths[2]
print(image_path)

read_divide_classify(image_path)

/home/michalis/Documents/Work/MELETI/inspection_client/temp_images/2-274-03-00788/0207_Main_frame_right.png


[(array([[234,  59, 171, ..., 236, 230, 228],
         [165,  95, 239, ..., 232, 231, 233],
         [122, 203, 236, ..., 222, 228, 225],
         ...,
         [237, 230, 240, ..., 241, 237, 236],
         [234, 233, 227, ..., 230, 230, 234],
         [238, 234, 227, ..., 232, 238, 236]], dtype=uint8),
  '0207',
  np.str_('sharp')),
 (array([[234, 237, 234, ..., 234, 232, 227],
         [227, 225, 229, ..., 224, 232, 229],
         [223, 223, 215, ..., 225, 223, 224],
         ...,
         [230, 231, 234, ..., 228, 233, 236],
         [237, 231, 233, ..., 231, 229, 236],
         [236, 231, 235, ..., 232, 232, 233]], dtype=uint8),
  '0207',
  np.str_('sharp')),
 (array([[230, 231, 228, ..., 224, 226, 232],
         [236, 227, 230, ..., 228, 222, 228],
         [231, 223, 226, ..., 223, 221, 220],
         ...,
         [232, 231, 228, ..., 229, 229, 226],
         [230, 234, 233, ..., 220, 229, 230],
         [234, 235, 234, ..., 234, 232, 226]], dtype=uint8),
  '0207',
  np.str_('sh

In [5]:
import os
import cv2
import numpy as np

class CollageHandler:
    def __init__(self, barcode):
        self.barcode = barcode
        self.blurry_batches = [[]]
        self.empty_batches = [[]]
        self.collage_dimensions = (160, 8)
        self.lock = threading.Lock()  # Add a lock for thread safety

    def get_current_blurry_batch_index(self):
        return len(self.blurry_batches) - 1

    def get_current_empty_batch_index(self):
        return len(self.empty_batches) - 1

    def save_batch_as_image(self, batch, patch_class, batch_index):
        folder_path = os.path.join(os.getcwd(), 'collages', self.barcode, patch_class)
        os.makedirs(folder_path, exist_ok=True)

        collage = self.create_collage(batch)
        file_path = os.path.join(folder_path, f'collage_{patch_class}_{batch_index}.png')

        cv2.imwrite(file_path, collage)

    def create_collage(self, batch):
        collage = np.zeros((150 * self.collage_dimensions[0], 150 * self.collage_dimensions[1]), dtype=np.uint8)
        
        max_width = 150
        max_height = 150

        for idx, (patch, page_number) in enumerate(batch):
            row = idx // 8
            col = idx % 8
            x_offset = col * max_width
            y_offset = row * max_height
            resized_patch = cv2.resize(patch, (max_width, max_height))
            collage[y_offset:y_offset+max_height, x_offset:x_offset+max_width] = resized_patch

        return collage

    def finish(self):
        for target, patch_class in zip([self.blurry_batches, self.empty_batches], ['blurry', 'empty']):
            if len(target[len(target) - 1]) > 0:
                self.save_batch_as_image(target[len(target) - 1], patch_class, len(target) - 1)


    def add_patch(self, labeled_patch):
        with self.lock:  # Ensure thread-safe access
            target = None
            currentIndex = None
            patch_class = labeled_patch[2]

            if patch_class == 'blurry':
                #print('blurry')
                target = self.blurry_batches
                currentIndex = self.get_current_blurry_batch_index
            elif patch_class == 'empty':
                #print('empty')
                target = self.empty_batches
                currentIndex = self.get_current_empty_batch_index

            if target is None:
                return 0  # Didn't add anything

            if len(target[currentIndex()]) >= self.collage_dimensions[0] * self.collage_dimensions[1]:
                collage_path = self.save_batch_as_image(target[currentIndex()], patch_class, currentIndex())
                # elongate
                target[currentIndex()] = collage_path
                target.append([])

            target[currentIndex()].append((labeled_patch[0], labeled_patch[1]))

            if patch_class == 'empty':
                return 1  # added empty

            return 2  # added blurry

        

    

In [10]:
collage_handler = CollageHandler(os.path.basename(folder_dir))

def process_file(file_name):
    if not file_name.endswith('.png'):
        return
    image_path = temp_dir + '/' + file_name
    labeled_patches = read_divide_classify(image_path)

    for labeled_patch in labeled_patches:
        collage_handler.add_patch(labeled_patch)

i = 0
with ProcessPoolExecutor() as executor:
    futures = [executor.submit(process_file, file_name) for file_name in os.listdir(temp_dir)]
    
    for future in futures:
        i += 1
        print(i)
        future.result()
    
    collage_handler.finish()


blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry
blurry

In [9]:
print(collage_handler.blurry_batches)

[[]]
